<a href="https://colab.research.google.com/github/dgalian/FTW_B7/blob/main/Capstone/Capstone_MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Market Basket Analysis to determine the typical combination of ICD Categories per Claim**

# Import libraries and load dataset

In [457]:
# Importing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [458]:
# Load claims breakdown by diagnosis
clm_icd = pd.read_csv('/content/CLM Breakdown csv.csv')

In [459]:
# Load ICD table
icd_table = pd.read_excel('/content/icd_table.xlsx')

# Data Prep
Profiling, Cleaning

In [460]:
icd_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4343 entries, 0 to 4342
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   icdno        4343 non-null   int64 
 1   icdcategory  4343 non-null   int64 
 2   icdgroup     4343 non-null   int64 
 3   ICD CODE     4343 non-null   object
 4   ICD DISEASE  4343 non-null   object
dtypes: int64(3), object(2)
memory usage: 169.8+ KB


In [461]:
# Creating basket of claims
basket = pd.merge(clm_icd, icd_table, on='icdno', how='left')

In [462]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382438 entries, 0 to 382437
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   cbdcode        382438 non-null  int64  
 1   claimcode      382438 non-null  int64  
 2   icdno          382438 non-null  int64  
 3   doctorcode     382438 non-null  int64  
 4   cptno          382438 non-null  int64  
 5   sacode         382438 non-null  int64  
 6   UNIT AMT       382438 non-null  float64
 7   QTY            382438 non-null  float64
 8   INCURRED AMT   382438 non-null  float64
 9   HOSP BILL AMT  382438 non-null  float64
 10  COVERED AMT    382438 non-null  float64
 11  BILL TO        382438 non-null  object 
 12  icdcategory    382355 non-null  float64
 13  icdgroup       382355 non-null  float64
 14  ICD CODE       382355 non-null  object 
 15  ICD DISEASE    382355 non-null  object 
dtypes: float64(7), int64(6), object(3)
memory usage: 49.6+ MB


In [463]:
basket.isnull().sum()

cbdcode           0
claimcode         0
icdno             0
doctorcode        0
cptno             0
sacode            0
UNIT AMT          0
QTY               0
INCURRED AMT      0
HOSP BILL AMT     0
COVERED AMT       0
BILL TO           0
icdcategory      83
icdgroup         83
ICD CODE         83
ICD DISEASE      83
dtype: int64

In [464]:
# Drop null values
basket = basket.dropna()
basket.isnull().sum()

cbdcode          0
claimcode        0
icdno            0
doctorcode       0
cptno            0
sacode           0
UNIT AMT         0
QTY              0
INCURRED AMT     0
HOSP BILL AMT    0
COVERED AMT      0
BILL TO          0
icdcategory      0
icdgroup         0
ICD CODE         0
ICD DISEASE      0
dtype: int64

In [465]:
basket['icdcategory'] = basket['icdcategory'].astype(int).astype(str)

In [466]:
basket = basket[['claimcode','icdno','icdcategory','ICD CODE','ICD DISEASE']]
basket.head()

,claimcode,icdno,icdcategory,ICD CODE,ICD DISEASE
0,203746,6127,84,G90.0,Idiopathic peripheral autonomic neuropathy
1,203913,40054,261,W55,Contact with other mammals
2,203913,40054,261,W55,Contact with other mammals
3,203913,40054,261,W55,Contact with other mammals
4,203913,40054,261,W55,Contact with other mammals


In [467]:
# Checking frequency of ICD categories
basket['icdcategory'].value_counts().sort_values(ascending=False)

277    52205
104    46489
61     21493
54     19404
0      16693
       ...  
200        1
281        1
183        1
28         1
73         1
Name: icdcategory, Length: 241, dtype: int64

In [468]:
# Checking number of ICD categories per claim
basket['claimcode'].value_counts().sort_values(ascending=False)

264685    50
241736    50
281217    49
226317    47
279115    46
          ..
404850     1
404851     1
404852     1
404854     1
658022     1
Name: claimcode, Length: 166947, dtype: int64

In [469]:
# Average number of ICD categories per claim
basket['claimcode'].value_counts().mean()

2.2902777528197573

**Checking the different baskets**

In [470]:
# Get list ICD Categories by Claim number
basket_bag = basket.groupby(['claimcode']).agg(lambda col: ','.join(col)).reset_index()

/usr/local/lib/python3.8/dist-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


In [471]:
store_items = basket_bag['icdcategory'].apply(lambda t: t.split(','))

In [472]:
store_items = list(store_items)

# Generating Association Rule

In [473]:
# Import permutations from the itertools module
from itertools import permutations

In [474]:
# Define the set of store items
flattened = [i for s in store_items for i in s]
store_items = list(set(flattened))

In [475]:
# Generate all possible rules from basket
rules = list(permutations(store_items, 2))
len(rules)

57840

# Support Metric

**One Hot Encoding**

In [476]:
# Import the transaction encoder function from mlxtend
from mlxtend.preprocessing import TransactionEncoder

# Instantiate transaction encoder and identify unique items in transactions
encoder = TransactionEncoder().fit(store_items)

# One-hot encode transactions
onehot = encoder.transform(store_items)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
print(onehot)

         0      1      2      3      4      5      6      7      8      9
0    False   True  False   True  False  False  False  False  False  False
1    False   True  False   True  False  False   True  False  False  False
2     True   True   True  False  False  False  False  False  False  False
3     True  False  False   True  False  False  False  False  False  False
4    False   True  False  False  False  False  False   True  False   True
..     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...
236  False   True  False  False  False  False   True   True  False  False
237  False   True   True  False   True  False  False  False  False  False
238  False   True   True  False  False   True  False  False  False  False
239  False   True   True  False  False  False  False  False  False  False
240  False  False   True   True  False  False  False  False  False  False

[241 rows x 10 columns]


**Compute the Support**

In [477]:
# Compute the support
support = onehot.mean()

# Print the support
print(support)

0    0.182573
1    0.502075
2    0.398340
3    0.203320
4    0.195021
5    0.186722
6    0.174274
7    0.170124
8    0.174274
9    0.174274
dtype: float64
